In [ ]:
''' 1. Module Import '''
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

In [ ]:
''' 2. 딥러닝 모델을 설계할 때 활용하는 장비 확인 '''
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

Using PyTorch version: 2.5.1  Device: cuda


In [ ]:
import torchvision.models as models
model = models.resnet34(pretrained = False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)
model = model.cuda()

/home/user1/miniconda3/envs/d2/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [29]:
BATCH_SIZE = 32
EPOCHS = 10

In [ ]:

''' 3. Data Augmentation이 적용된 CIFAR10 데이터 다운로드 (Train set, Test set 분리하기) '''
train_dataset = datasets.CIFAR10(root = "../data/CIFAR_10",
                                  train = True,
                                  download = True,
                                  transform = transforms.Compose([
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))

test_dataset = datasets.CIFAR10(root = "../data/CIFAR_10",
                                train = False,
                                transform = transforms.Compose([
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                            batch_size = BATCH_SIZE,
                                            shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = BATCH_SIZE,
                                          shuffle = False)

Files already downloaded and verified


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
    
    test_loss /= (len(test_loader.dataset) / BATCH_SIZE)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [ ]:
for epoch in range(1, EPOCHS + 1):
    # train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} %".format(epoch, test_loss, test_accuracy))


[EPOCH: 1], 	Test Loss: 2.4450, 	Test Accuracy: 10.58 % 


[EPOCH: 2], 	Test Loss: 2.4437, 	Test Accuracy: 10.69 % 


[EPOCH: 3], 	Test Loss: 2.4446, 	Test Accuracy: 10.47 % 


[EPOCH: 4], 	Test Loss: 2.4449, 	Test Accuracy: 10.55 % 


[EPOCH: 5], 	Test Loss: 2.4444, 	Test Accuracy: 10.60 % 


[EPOCH: 6], 	Test Loss: 2.4435, 	Test Accuracy: 11.12 % 


[EPOCH: 7], 	Test Loss: 2.4441, 	Test Accuracy: 10.65 % 


[EPOCH: 8], 	Test Loss: 2.4449, 	Test Accuracy: 10.77 % 


[EPOCH: 9], 	Test Loss: 2.4439, 	Test Accuracy: 10.75 % 


[EPOCH: 10], 	Test Loss: 2.4448, 	Test Accuracy: 10.36 % 



In [34]:
''' 8. 불러온 Resnet34 모델 학습을 진행하며 학습 데이터에 대한 모델 성능을 확인하는 함수 정의 '''
def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
                loss.item()))


In [35]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} %".format(epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/50000 (0%)]	Train Loss: 2.666944
Train Epoch: 1 [6400/50000 (13%)]	Train Loss: 1.584967
Train Epoch: 1 [12800/50000 (26%)]	Train Loss: 1.649621
Train Epoch: 1 [19200/50000 (38%)]	Train Loss: 1.393158
Train Epoch: 1 [25600/50000 (51%)]	Train Loss: 1.799635
Train Epoch: 1 [32000/50000 (64%)]	Train Loss: 1.554136
Train Epoch: 1 [38400/50000 (77%)]	Train Loss: 1.392786
Train Epoch: 1 [44800/50000 (90%)]	Train Loss: 1.111435

[EPOCH: 1], 	Test Loss: 1.2170, 	Test Accuracy: 57.86 %
Train Epoch: 2 [0/50000 (0%)]	Train Loss: 1.279277
Train Epoch: 2 [6400/50000 (13%)]	Train Loss: 0.887981
Train Epoch: 2 [12800/50000 (26%)]	Train Loss: 1.220961
Train Epoch: 2 [19200/50000 (38%)]	Train Loss: 1.295018
Train Epoch: 2 [25600/50000 (51%)]	Train Loss: 1.472755
Train Epoch: 2 [32000/50000 (64%)]	Train Loss: 0.710842
Train Epoch: 2 [38400/50000 (77%)]	Train Loss: 0.923910
Train Epoch: 2 [44800/50000 (90%)]	Train Loss: 0.941077

[EPOCH: 2], 	Test Loss: 0.9454, 	Test Accuracy: 66.47 %
Tr